In [1]:
"""
PART ONE OF TWO (Part two will be normalization 
Objective: Input a companies product page and get all the desired links. Than with the use of OpenAI, retrieve the data points. 
Emphasis on reusability.

Currently focusing on just HP for proof of concept.

Soon I would like the elements that are used for loading more products, going to another page and selecting all products to be 
more autonomously acquired using OpenAI as well as being intergrated with the program.

"""

'\nPART ONE OF TWO\nObjective: Input a companies product info and get all the desired data points out of it. Emphasis on reusability.\n\nNote: This is a combination of part one and two from any earlier version of the program. Currently focusing on just HP.\n\nSummary\n1. \n\n'

In [1]:
import csv
import os,json,ast
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from bs4 import BeautifulSoup
from datetime import datetime
from openai import OpenAI
from urllib.parse import urlparse


In [2]:
class FilesManager:
    def __init__(self):
        self.api_key_directory = None
        self.output_directory = None

    def set_directory(self, prompt_message):
        """Prompts the user to enter a directory path and validates it."""
        while True:
            # Changed for testing
            #directory = input(prompt_message)
            directory = r'C:\Users\keena\Desktop\Comp Sys Project'
            if os.path.isdir(directory):
                print("The directory is valid.")
                return directory
            else:
                print("The directory is invalid. Please try again.")

    def display_sample_api_key(self):
        """Displays sample API key formats for JSON and text files."""
        print("\nSample API Key Formats:")
        print("JSON file format:")
        print('{\n    "key": "sk-xxxxxx...xxxxx"\n}')
        print("\nText file format:")
        print("sk-xxxxxx...xxxxx\n")

    def get_openai_api_key(self):
        """Lists files in the API key directory, allows user to select one or enter the key manually."""
        while True:
            files = [f for f in os.listdir(self.api_key_directory) if os.path.isfile(os.path.join(self.api_key_directory, f))]
            
            if not files:
                self.display_sample_api_key()
                print("No files found in the directory. Please enter the API key manually.")
                api_key = input("Enter the OpenAI API key: ")
                return api_key
            
            print("Files found in the directory:")
            for idx, file in enumerate(files):
                print(f"{idx + 1}. {file}")

            print(f"{len(files) + 1}. Enter API key manually")
            self.display_sample_api_key()

            # Changed for testing
            #choice = input(f"Select the file number (1-{len(files) + 1}): ")
            choice = 1

            
            try:
                choice = int(choice)
                if choice == len(files) + 1:
                    api_key = input("Enter the OpenAI API key: ")
                    return api_key
                elif 1 <= choice <= len(files):
                    selected_file = os.path.join(self.api_key_directory, files[choice - 1])
                    file_extension = os.path.splitext(selected_file)[1]

                    if file_extension == '.txt':
                        with open(selected_file, 'r') as file:
                            api_key = file.read().strip()
                            return api_key
                    elif file_extension == '.json':
                        with open(selected_file, 'r') as file:
                            data = json.load(file)
                            api_key = data.get("key", "").strip()
                            if api_key:
                                return api_key
                            else:
                                print("JSON file does not contain a valid 'key'. Please try again.")
                    else:
                        print("Unsupported file format. Please select a .txt or .json file.")
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    def set_output_directory(self):
        """Prompts the user to enter an output directory, validates it, and creates a new folder for data output."""
        while True:
            # Change for testing
            #directory = input("Please enter the directory path where data should be outputted to: ")
            directory = r'C:\Users\keena\Desktop\Comp Sys Project'

            if os.path.isdir(directory):
                print("The directory is valid.")
                new_folder = os.path.join(directory, datetime.now().strftime('%Y%m%d_%H%M%S'))
                os.makedirs(new_folder)
                print(f"New folder created for data output: {new_folder}")
                return new_folder
            else:
                print("The directory is invalid. Please try again.")
                
    def iter_in_chunks(self, lst, chunk_size):
        """This is used for sending just groups of urls to OpenAI to avoid sending too much at once."""
        for i in range(0, len(lst), chunk_size):
            yield lst[i:i + chunk_size]
            
    def write_links_to_csv(self, company_name, output_directory, all_csv_data):
        """Write the collected links to a CSV file."""
        current_date = datetime.now().strftime('%d%b%Y').upper()
        csv_file_name = f"{company_name}_{current_date}.txt"
        csv_file_path = os.path.join(output_directory, csv_file_name)
    
        with open(csv_file_path, mode='a', newline='') as file:
            file.write(all_csv_data)
        return csv_file_path

In [3]:
class SeleniumManager:
    def __init__(self):
        self.driver = None
    
    def initialize_webdriver(self):
        options = Options()
        # options.add_argument('--headless')  # Optionally run in headless mode
        options.add_argument('--no-sandbox')  # Bypass OS security model
        options.add_argument('--disable-gpu') 
        options.add_argument('start-maximized')  
        options.add_argument('disable-infobars')
        options.add_argument('--disable-extensions')
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


    def load_page(self, url):
        if self.driver is None:
            self.initialize_webdriver()
        self.driver.get(url)
        sleep(5)
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

    #############NOT UNIVERSAL#############
    def dismiss_popup(self):
        try:
            no_thanks_button = WebDriverWait(self.driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.optly-country-continue[data-optly-event="e_continue_country"]'))
            )
            no_thanks_button.click()
        except Exception as e:
            print("No 'NO THANKS' button found or error: ", str(e))
    ########################################
    
    def scroll_page(self):
        """Scroll the page down by 500 pixels."""
        self.driver.execute_script("window.scrollBy(0, 250);")
        sleep(2)  # Wait for the page to load more items

    def scroll_page_up(self):
        """Scroll the page up by 100 pixels."""
        self.driver.execute_script("window.scrollBy(0, -500);")
        # sleep(.5)  # Wait for the page to adjust

    #############NOT UNIVERSAL#############
    def collect_links(self, base_url, existing_urls):
        """Collect product links from the page and return them."""
        links = self.driver.find_elements(By.CSS_SELECTOR, 'a.ProductTitle-module_link__3V6Pz')
        complete_url = []
        for link in links:
            href = link.get_attribute('href')
            if "shop/pdp" in href and href not in existing_urls and not href.endswith("#reviews"):
                full_url = base_url + href
                # For testing purposes
                #print(full_url)
                existing_urls.add(href)
                complete_url.append(full_url)
        return complete_url
    ########################################

    #############NOT UNIVERSAL#############
    def handle_load_more(self):
        """Handle clicking the 'Load More' button if it is present."""
        try:
            load_more_button = WebDriverWait(self.driver, 5).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, 'button[data-gtm-value="load-more"]'))
            )
            self.driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
            load_more_button.click()
            print('---------------------MORE PRODUCTS LOADED---------------------')
            return True
        except:
            print("No more 'Load More' button found or error.")
            return False
    ########################################

    def quit_driver(self):
        if self.driver is not None:
            self.driver.quit()

In [4]:
class BeautifulSoupManager:
    def __init__(self, page_source):
        self.soup = BeautifulSoup(page_source, 'html.parser')

    #############NOT UNIVERSAL#############
    def collect_links(self, base_url, existing_urls):
        """Collect product links from the page and return them."""
        links = self.soup.select('a.ProductTitle-module_link__3V6Pz')
        new_links = []
        for link in links:
            href = link.get('href')
            if "shop/pdp" in href and href not in existing_urls and not href.endswith("#reviews"):
                if href.startswith("http"):
                    complete_url = href
                else:
                    complete_url = base_url + href
                print(complete_url)
                existing_urls.add(href)
                new_links.append(complete_url)
        return new_links
    ########################################



In [5]:
class OpenAIManager:
    def __init__(self, api_key, fileMgr):
        self.api_key = api_key
        self.fileMgr = fileMgr
        
    #############NOT UNIVERSAL#############
    def generate_csv_data_from_urls(self, urlList):
        data_points_list = ''
        total_cost = 0
        chunk_size = 10
    
        for url_set in self.fileMgr.iter_in_chunks(urlList, chunk_size):
            print(f'----------NEW SET TEST----------')
            print(url_set)
            
            prompt = f"""Generate csv data for each provided link with the following columns:
    url,company,products,model,product_line,price,discounted_price,cpu_manufacturer,cpu_model,cpu_cores,cpu_threads,cpu_base_clock_speed,cpu_turbo_clock_speed,gpu_manufacturer,gpu_model,gpu_vram,ram_capacity,ram_type,ram_speed,storage_type,storage_capacity,display_size,display_resolution,display_refresh_rate,display_panel_type,dimensions,weight,material,battery_capacity,battery_life,power_supply,usb_ports,hdmi_displayport,audio_ports,network_connectivity,operating_system,pre_installed_software,keyboard,touchpad,security_features,webcam,speakers,cooling_system,warranty,customer_support
    
    Example Output:
    "https://www.hp.com/us-en/shop/pdp/hp-elitebook-865-16-inch-g10-notebook-pc-wolf-pro-security-edition-p-8f838ua-aba-1","HP","EliteBook 865 16-inch G10 Notebook PC Wolf Pro Security Edition","865 G10","EliteBook",N/A,N/A,AMD,Ryzen 7 PRO 7850U,8,16,N/A,4.7 GHz,AMD,Radeon Graphics,Integrated,"16 GB",LPDDR5,N/A,SSD,"512 GB","16 inches","1920 x 1200 (WUXGA)",N/A,IPS,"14.07 x 9.22 x 0.79 inches","4.11 pounds",N/A,"68 WHr",N/A,"65W USB-C Adapter","2 x USB-C, 2 x USB-A","1 x HDMI 2.0","1 x Headphone/Microphone Combo","Wi-Fi 6E, Bluetooth 5.2",Windows 11 Pro,"HP Wolf Security, HP QuickDrop","Backlit, Spill-resistant","Multi-touch Gesture Support","Fingerprint Reader, IR Camera","720p HD IR","Dual Stereo Speakers","N/A","3 Year Limited Warranty","24/7 Support"
    
    Link:
    {url_set}
    If data point not available, use N/A. No other text. No newlines('\\n') and no ```csv / ```. Use '|' to differentiate between products and one at the end in case more data is added.
    """
    
            client = OpenAI()
            
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "system", "content": prompt}],
                max_tokens=3000
            )
            
            data = response.choices[0].message.content.strip()
            data_points_list += data
            tokens_used = response.usage.total_tokens
            prompt_tokens = response.usage.prompt_tokens
            completion_tokens = response.usage.completion_tokens
            
            prompt_cost = prompt_tokens * 5 / 1000000  # GPT-4o input cost
            input_cost = completion_tokens * 15 / 1000000  # GPT-4o output cost
            
            current_total_cost = prompt_cost + input_cost
            total_cost += current_total_cost 
            
            print(f"Link: {url_set}\nData:\n{data}\nTokens Used: {tokens_used}\nCost: ${current_total_cost}")
        
        return data_points_list
    ########################################

    def def_product_element(self):
        pass
    def def_more_products_element(self):
        pass

In [6]:
def main():
    # Setting up the different classes
    fileMgr = FilesManager()
    seleniumMgr = SeleniumManager()
    
    # Setting up the directory where data is located and will be outputted to.
    fileMgr.api_key_directory = fileMgr.set_directory("Please enter the directory path where the API key is located: ")
    api_key = fileMgr.get_openai_api_key()
    openaiMgr = OpenAIManager(api_key=api_key, fileMgr=fileMgr)
    
    fileMgr.output_directory = fileMgr.set_output_directory()
    print(f"Data will be outputted to: {fileMgr.output_directory}")

    # Initialize the WebDriver
    seleniumMgr.initialize_webdriver()
    driver = seleniumMgr.driver

    # Change for testing
    #url = input("Please enter the product list URL (e.g., https://www.hp.com/us-en/shop/vwa/laptops?jumpid=ma_lt_featured_na_1_240417): ")
    url = 'https://www.hp.com/us-en/shop/vwa/laptops/Price=1-500;form=Convertible;segm=Home?jumpid=ma_lt_featured_na_3_231108'

    # For Testing
    #company_name = input("Please enter the company name: ")
    company_name = "HP"

    # This was for getting the base html but no longer useful.
    parsed_url = urlparse(url)
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
    

    existing_urls = set()
    all_links = []
    
    try:
        # Load the page and handle the popup
        seleniumMgr.load_page(url)
        seleniumMgr.dismiss_popup()

        # Scroll the page and collect links
        while True:
            seleniumMgr.scroll_page()
            sleep(1)
            if not seleniumMgr.handle_load_more():
                break  # Exit the loop if no "Load More" button is found or there's an error

        # Scroll back up to double-check for any missed links
        while True:
            seleniumMgr.scroll_page_up()
            sleep(.05)
            if seleniumMgr.driver and seleniumMgr.driver.execute_script("return window.pageYOffset;") == 0:
                break  # Stop scrolling up when we reach the top of the page
        
        # Extract the final page source
        page_source = driver.page_source

        # Use BeautifulSoupManager to parse the page and collect links
        soupMgr = BeautifulSoupManager(page_source)
        #all_links = seleniumMgr.collect_links(base_url, existing_urls)
        all_links = soupMgr.collect_links(base_url, existing_urls)

        
        print(f'{len(all_links)} Product Links Collected')
            
    except Exception as e:
        print("Error during page interaction: ", str(e))
        
    finally:
        # Close the driver
        seleniumMgr.quit_driver()

    # Send each link in csv to openai to populate the rest of the data points
    print('Product links collected.')
    print('Initializing OpenAI data points collecting...')

    # OpenAI section to get the data points
    all_csv_data = openaiMgr.generate_csv_data_from_urls(all_links)

    print('OpenAI data points collected...')
    print('Writing data to text file...')
    # CHANGE THIS TO WRITE ALL DATA POINTS TO CSV
    # Write links to CSV
    fileMgr.write_links_to_csv(company_name, fileMgr.output_directory, all_csv_data)
    print('Data written to text file...')
    print('PROGRAM COMPLETE')

In [7]:
if __name__ == "__main__":
    main()

The directory is valid.
Files found in the directory:
1. ChatGPTAPIKey.json
2. Detailed Plan for Computer Systems Comparison Project.docx
3. HP_product_data_points.txt
4. ML Comp Sys Data.csv
5. Output.txt
6. Project Issues.docx
7. PtOneCompSysScrape.py
8. TEST_product_links.txt
9. Enter API key manually

Sample API Key Formats:
JSON file format:
{
    "key": "sk-xxxxxx...xxxxx"
}

Text file format:
sk-xxxxxx...xxxxx

The directory is valid.
New folder created for data output: C:\Users\keena\Desktop\Comp Sys Project\20240708_100151
Data will be outputted to: C:\Users\keena\Desktop\Comp Sys Project\20240708_100151
No more 'Load More' button found or error.
https://www.hp.com/us-en/shop/pdp/hp-pavilion-x360-convertible-15t-dw400-4v223av-1
https://www.hp.com/us-en/shop/pdp/hp-pavilion-x360-2-in-1-laptop-14t-ek1000-14-742j3av-1
https://www.hp.com/us-en/shop/pdp/hp-chromebook-plus-14-x360-laptop-14ct-cd000-4v272av-1
https://www.hp.com/us-en/shop/pdp/hp-chromebook-x360-133-laptop-14at-ca100-